<a href="https://colab.research.google.com/github/carlibeisel/Drains_Lower_Boise_River/blob/main/05_compile_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: July 21, 2022

Date modified: May 13, 2024

Purpose: Merges flow, climate, and land use annual stats together.

*** cannot use direct output from this in R. It merges Mason Creek and Mason Drain wrong. This was manually corrected.

In [1]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd # to work with dataframe
import os # for file paths
import glob # read in a folder of csv
import numpy as np # basic statistics

#connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
## ------------------------------ ##
## IMPORT ALL DATASETS ##
## ------------------------------ ##

drains = pd.read_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/drainage_flows_out/annual_flow_vals.csv')

land_files = sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/final_metrics/*.csv'))
land = []
for i in land_files:
  land.append(pd.read_csv(i))
land = pd.concat(land)

clim_files = sorted(glob.glob('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/final/*.csv'))
clim = []
for i in clim_files:
  clim.append(pd.read_csv(i))
clim = pd.concat(clim)

hydromet = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_output/hydromet_data_out/model_input_hydromet.csv')

In [6]:
## ------------------------------ ##
## Use a dictionary to help align ##
## ------------------------------ ##

relate = pd.read_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drainage_flows/dain_relates.csv')
spatial = relate.dropna(subset=['Spatial Name'])
newnames = dict(zip(spatial['Spatial Name'], spatial['NewName']))

clim['NewName'] = clim['NAME'].map(newnames)
clim = clim.drop(['Unnamed: 0', 'NAME'], axis=1)
land['NewName2'] = land['DrainName'].map(newnames)
land = land.drop(['Unnamed: 0', 'DrainName'], axis=1)

In [8]:
## ------------------------------ ##
## Merge the files together ##
## ------------------------------ ##

merge2 = drains.merge(clim, left_on = ['Year', 'Name'], right_on = ['Year', 'NewName'])
merge3 = merge2.merge(land, left_on = ['Year', 'Name'], right_on = ['dates', 'NewName2'])
merge3 = merge3.drop(['Unnamed: 0', 'NewName', 'dates',  'NewName2'], axis=1)


merge_final = pd.merge(merge3, hydromet, on='Year', how='left')


In [9]:
merge_final.columns

Index(['Name', 'Year', 'Sum_AF', 'SD_AF', 'ant_prcp', 'wy_prcp', 'irrig_temp',
       'JuneAug_temp', 'et', 'class1_urban', 'class2_crops', 'contagion',
       'largest_patch_index', 'Unnamed: 0', 'LP_inflows', 'Max_Fill',
       'Carryover'],
      dtype='object')

In [10]:
## ------------------------------ ##
##        Export the file         ##
## ------------------------------ ##
merge_final.to_csv('/content/gdrive/MyDrive/Data/Drains_Lower_Boise_River/data_output/compile_data_out/model_input.csv')